# Snowflake + Dask

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="300">
        </td>
    </tr>
</table>

This tutorial describes how to connect to Snowflake, load data into a Snowflake table, and work with that data in Dask.

<hr>

## Connect to Snowflake

This example uses data stored in a Snowflake data warehouse that is managed by the team at Saturn Cloud. We've set up a read-only user for use in these examples. If you would like to access data stored in your own Snowflake account, see the [README](./README.md).

In [ ]:
import os
import snowflake.connector
import pandas as pd

conn_info = {
    "account": os.environ["EXAMPLE_SNOWFLAKE_ACCOUNT"],
    "user": os.environ["EXAMPLE_SNOWFLAKE_USER"],
    "password": os.environ["EXAMPLE_SNOWFLAKE_PASSWORD"],
    "database": os.environ["TAXI_DATABASE"],
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Set up a query template

We need to set up a query template containing a bind variable that will result in Dask issuing multiple queries that each extract a slice of the taxi data based on the pickup_datetime column. These slices will become our partitions in a Dask dataframe. We use a [binding for the Snowflake query](https://docs.snowflake.com/en/user-guide/python-connector-example.html#binding-data) so that we can pass different date values at execution time.

In [ ]:
query = """
SELECT *
FROM taxi_yellow
WHERE
    date(pickup_datetime) = '{day}'
"""

Validate the query is good with pandas

In [ ]:
df = pd.read_sql(query.format(day="2019-01-01"), conn)
len(df), df.memory_usage().sum() / 1e6  # memory size in MB

<hr>

## Initialize Dask cluster

This tutorial uses multiple machines to show how to work with large datasets using Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='large'` --> each worker machine will have 2 CPU cores, 16GB of RAM

To see a list of possible sizes, run the code below.

In [ ]:
import dask_saturn

dask_saturn.describe_sizes()

The `dask-saturn` code below creates two important objects: a cluster and a client.

* `cluster`: knows about and manages the scheduler and workers
    - can be used to create, resize, reconfigure, or destroy those resources
    - knows how to communicate with the scheduler, and where to find logs and diagnostic dashboards
* `client`: tells the cluster to do things
    - can send work to the cluster
    - can restart all the worker processes
    - can send data to the cluster or pull data back from the cluster

In [ ]:
import dask
from dask.distributed import Client
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers, scheduler_size="medium", worker_size="large", nthreads=2
)
client = Client(cluster)
cluster

If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

<hr>

## Load larger data with Dask!

We set up a function with `dask.delayed`. `@delayed` is a decorator that turns a Python function into a function suitable for running on the Dask cluster. When you execute a delayed function, instead of executing the operation, it returns a delayed result that represents what the return value of the function will be. `dask.dataframe.from_delayed` takes a list of these delayed objects, and concatenates them into a Dask dataframe.

In [ ]:
import dask.dataframe as dd

In [ ]:
print(query)

In [ ]:
@dask.delayed
def load(conn_info, query, day):
    with snowflake.connector.connect(**conn_info) as conn:
        return pd.read_sql(query.format(day=day), conn)

In [ ]:
out = load(conn_info, query, "2019-01-01")
out

We can call `compute()` to execute the function and see the output (in this case a Pandas dataframe)

In [ ]:
type(out.compute())

Now, let's load more days using Dask! First we want to pull a range of dates where know data exists. We can run a quick Snowflake query for that

In [ ]:
date_query = """
SELECT
    DISTINCT(DATE(pickup_datetime)) as date
FROM taxi_yellow
WHERE
    pickup_datetime BETWEEN '2019-01-01' and '2019-01-31'
"""
dates_df = pd.read_sql(date_query, conn)
dates = dates_df["DATE"].tolist()
dates[:5]

Then, we build up a list of delayed objects that call the `load()` function we created

In [ ]:
delayed_obs = [load(conn_info, query, day) for day in dates]
delayed_obs[:5]

Finally, create a Dask Dataframe!

In [ ]:
ddf = dd.from_delayed(delayed_obs)
ddf

Notice that the above command ran pretty quickly. This is because Dask only executes the task graph when you perform certain actions, such as writing a file or getting the `len` of the DataFrame

In [ ]:
len(ddf)

<br>

We can use `repartition()` to introduce more parallelism. This helps downstream processes execute faster by splitting the work across more cores.

In [ ]:
ddf = ddf.repartition(npartitions=100)
ddf

In [ ]:
len(ddf)

<br>
The cell below will execute the Snowflake queries across the cluster, compute the row count and size of each partition in parallel, and then aggregate the results to present the row count and size of the entire Dask dataframe.

In [ ]:
print(f"Num rows: {len(ddf)}, Size: {ddf.memory_usage(deep=True).sum().compute() / 1e6} MB")

The partitions in the Dask dataframe are pandas dataframes

In [ ]:
ddf_part = ddf.partitions[0].compute()
type(ddf_part)

If we plan on performing a lot of operations using this Dask dataframe (such as training a machine learning model), and the data will fit in the memory of the _cluster_, we should `persist()` the dataframe to perform all the loading up-front.

In [ ]:
from dask.distributed import wait

ddf = ddf.persist()
_ = wait(ddf)

The following cell should execute much faster than previously, because all the data is loaded into memory

In [ ]:
print(f"Num rows: {len(ddf)}, Size: {ddf.memory_usage(deep=True).sum().compute() / 1e6} MB")

<hr>

## Next Steps

In this tutorial, you learned how to use Snowflake and `snowflake-connector-python` to execute SQL queries over large datasets. You also learned how to read those query results into a Dask DataFrame.

Next, try [this dask-ml notebook](../nyc-taxi-snowflake/hyperparameter-dask.ipynb) to see how to use this Dask DataFrame to accelerate common machine learning tasks like feature engineering and hyperparameter tuning.

<hr>